# Setting up the Chromedriver

In [ ]:
!pip install scrapy
!pip install selenium
!pip install selenium-stealth
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Script for France Region

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from scrapy import Selector
from selenium_stealth import stealth
import os
import csv
import time


FIELD_NAMES = ['Name', 'Type', 'Address', 'Phone', 'Location du lieu', 'Invites', 'Espaces', 'Services', 'Logement', 'Quel type de réception', 'Est-il possible de louer', 'une limite horaire à respecter pour', 'Le traiteur est-il', 'Existe-t-il un prix fixe pour', 'effectue le paiement', 'Source Url', 'Base Url']
FILE_NAME = './mariagesFrance.csv'
urlList = []
pageCntr = 1

def writeCSV(data, fieldName, FILE_NAME):
    fileExists = os.path.isfile(FILE_NAME)
    with open(FILE_NAME, 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldName, lineterminator='\n')
        if not fileExists:
            writer.writeheader()
        writer.writerow(data)

chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory" : "/some/path"}
# chromeOptions.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

driver.get("https://www.mariages.net/domaine-mariage")
time.sleep(2)

try:
  cookiesBtnElem = driver.find_element_by_xpath("//button[text()='Accepter']")
  driver.execute_script("arguments[0].click()", cookiesBtnElem)
  time.sleep(1)
except:
  pass

while True:
  pageCntr += 1
  html = driver.page_source
  respObj = Selector(text=html)

  #if pageCntr > 27:
  cards = respObj.xpath("//div[@data-list-type='Catalog']/div[@id]")
  for card in cards:
    urlList.append(card.xpath(".//a[contains(@id, 'app_lnk')]/@href").get())

  nextPageType1 = respObj.xpath(f"//a[@data-page and text()='{pageCntr}']")
  nextPageType2 = respObj.xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
  
  if nextPageType1:
    nextBtnElem = driver.find_element_by_xpath(f"//a[@data-page and text()='{pageCntr}']")
    driver.execute_script("arguments[0].click()", nextBtnElem)
    time.sleep(2)
    print(f"\n\n PAGE-{pageCntr}")
  elif nextPageType2:
    nextBtnElem = driver.find_element_by_xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
    driver.execute_script("arguments[0].click()", nextBtnElem)
    time.sleep(2)
    print(f"\n\n PAGE-{pageCntr}")
  else:
    break

for url in urlList:
  driver.get(url)
  WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//h1")))

  try:
    phoneBtnElem = driver.find_element_by_xpath("//span[@class='app-emp-phone-txt']")
    driver.execute_script("arguments[0].click()", phoneBtnElem)
    WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//i[contains(@class, 'phone')]/parent::p")))
  except:
    pass

  html = driver.page_source
  respObj = Selector(text=html)

  quelType = respObj.xpath("//span[contains(text(), 'Quel type de réception')]/parent::li/div/div/div/text()").getall()
  data = {
      FIELD_NAMES[0]: respObj.xpath("normalize-space(//h1/text())").get(),
      FIELD_NAMES[1]: respObj.xpath("//h1/following-sibling::i/@class").get(),
      FIELD_NAMES[2]: respObj.xpath("normalize-space((//div[contains(@class, 'address')]/text())[2])").get(),
      FIELD_NAMES[3]: f'''{respObj.xpath("normalize-space(//i[contains(@class, 'phone')]/parent::p/text()[last()])").get()}''',
      FIELD_NAMES[4]: respObj.xpath("normalize-space(//span[text()='Location du lieu']/following-sibling::span/text())").get(),
      FIELD_NAMES[5]: respObj.xpath('''normalize-space(//span[text()="Nº d'invités "]/following-sibling::span/text())''').get(),
      FIELD_NAMES[6]: respObj.xpath("normalize-space(//span[text()='Espaces']/parent::div/text()[last()])").get(),
      FIELD_NAMES[7]: respObj.xpath("normalize-space(//span[text()='Services']/parent::div/text()[last()])").get(),
      FIELD_NAMES[8]: respObj.xpath("normalize-space(//span[text()='Logement']/parent::div/text()[last()])").get(),
      FIELD_NAMES[9]: ", ".join(i.strip() for i in quelType),
      FIELD_NAMES[10]: respObj.xpath("normalize-space(//span[contains(text(), 'Est-il possible de louer')]/parent::li/div/text())").get(),
      FIELD_NAMES[11]: respObj.xpath("normalize-space(//span[contains(text(), 'une limite horaire à respecter pour')]/parent::li/div/text())").get(),
      FIELD_NAMES[12]: respObj.xpath("normalize-space(//span[contains(text(), 'Le traiteur est-il')]/parent::li/div/text())").get(),
      FIELD_NAMES[13]: respObj.xpath("normalize-space(//span[contains(text(), 'Existe-t-il un prix fixe pour')]/parent::li/div/text())").get(),
      FIELD_NAMES[14]: respObj.xpath("normalize-space(//span[contains(text(), 'effectue le paiement')]/parent::li/div/text())").get(),
      FIELD_NAMES[15]: driver.current_url,
      FIELD_NAMES[16]: "https://www.mariages.net/domaine-mariage"

  }
  print(data)
  writeCSV(data, FIELD_NAMES, FILE_NAME)
  data.clear()



# Script for Spain Region

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from scrapy import Selector
from selenium_stealth import stealth
import os
import csv
import time


FIELD_NAMES = ['Name', 'Type', 'Address', 'Phone', 'Precio por menú', 'Nº de invitados', 'Espacios', 'Servicios', 'Alojamiento', '¿Qué tipo de cocina sirves?', 'Es posible alquilar únicamente el espacio', 'Tienes límite de hora en las celebraciones', 'Se grava el servicio de catering', 'Existe algún pago fijo por el lugar', 'Cómo se efectúa el pago', 'Source Url']
BASE_URLS = [
             'https://www.bodas.net/bodas/banquetes/fincas',
             'https://www.bodas.net/bodas/banquetes/hoteles',
             'https://www.bodas.net/bodas/banquetes/restaurantes',
             'https://www.bodas.net/bodas/banquetes/castillos',
             'https://www.bodas.net/bodas/banquetes/cortijos',
             'https://www.bodas.net/bodas/banquetes/haciendas',
             'https://www.bodas.net/bodas/banquetes/bodegas',
             'https://www.bodas.net/bodas/banquetes/espacios-singulares',
             'https://www.bodas.net/bodas/banquetes/bodas-en-la-playa',
]

def writeCSV(data, fieldName, FILE_NAME):
    fileExists = os.path.isfile(FILE_NAME)
    with open(FILE_NAME, 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldName, lineterminator='\n')
        if not fileExists:
            writer.writeheader()
        writer.writerow(data)

chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory" : "/some/path"}
# chromeOptions.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

for baseUrl in BASE_URLS:
  urlList = []
  pageCntr = 1
  driver.get(baseUrl)
  time.sleep(2)

  try:
    cookiesBtnElem = driver.find_element_by_xpath("//button[text()='Accepter']")
    driver.execute_script("arguments[0].click()", cookiesBtnElem)
    time.sleep(1)
  except:
    pass

  while True:
    pageCntr += 1
    html = driver.page_source
    respObj = Selector(text=html)

    #if pageCntr > 27:
    cards = respObj.xpath("//div[@data-list-type='Catalog']/div[@id]")
    for card in cards:
      urlList.append(card.xpath(".//a[contains(@id, 'app_lnk')]/@href").get())

    nextPageType1 = respObj.xpath(f"//a[@data-page and text()='{pageCntr}']")
    nextPageType2 = respObj.xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
    
    if nextPageType1:
      nextBtnElem = driver.find_element_by_xpath(f"//a[@data-page and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    elif nextPageType2:
      nextBtnElem = driver.find_element_by_xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    else:
      break

  for url in urlList:
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//h1")))

    try:
      phoneBtnElem = driver.find_element_by_xpath("//span[@class='app-emp-phone-txt']")
      driver.execute_script("arguments[0].click()", phoneBtnElem)
      WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//i[contains(@class, 'phone')]/parent::p")))
    except:
      pass

    html = driver.page_source
    respObj = Selector(text=html)

    quelType = respObj.xpath("//span[contains(text(), 'de cocina sirves')]/parent::li/div/div/div/text()").getall()
    prem = respObj.xpath("//h1/following-sibling::i/@class").get()
    try:
      if "premium" in prem:
        prem = "Premium"
    except:
      pass

    data = {
        FIELD_NAMES[0]: respObj.xpath("normalize-space(//h1/text())").get(),
        FIELD_NAMES[1]: prem,
        FIELD_NAMES[2]: respObj.xpath("normalize-space((//div[contains(@class, 'address')]/text())[2])").get(),
        FIELD_NAMES[3]: f'''{respObj.xpath("normalize-space(//i[contains(@class, 'phone')]/parent::p/text()[last()])").get()}''',
        FIELD_NAMES[4]: respObj.xpath("normalize-space(//span[text()='Precio por menú']/following-sibling::span/text())").get(),
        FIELD_NAMES[5]: respObj.xpath('''normalize-space(//span[text()="Nº de invitados"]/following-sibling::span/text())''').get(),
        FIELD_NAMES[6]: respObj.xpath("normalize-space(//span[text()='Espacios']/parent::div/text()[last()])").get(),
        FIELD_NAMES[7]: respObj.xpath("normalize-space(//span[text()='Servicios']/parent::div/text()[last()])").get(),        
        FIELD_NAMES[8]: respObj.xpath("normalize-space(//span[text()='Alojamiento']/parent::div/text()[last()])").get(),
        FIELD_NAMES[9]: ", ".join(i.strip() for i in quelType),        
        FIELD_NAMES[10]: respObj.xpath("normalize-space(//span[contains(text(), 'Es posible alquilar únicamente el espacio')]/parent::li/div/text())").get(),
        FIELD_NAMES[11]: respObj.xpath("normalize-space(//span[contains(text(), 'Tienes límite de hora en las celebraciones')]/parent::li/div/text())").get(),
        FIELD_NAMES[12]: respObj.xpath("normalize-space(//span[contains(text(), 'grava el servicio de catering')]/parent::li/div/text())").get(),
        FIELD_NAMES[13]: respObj.xpath("normalize-space(//span[contains(text(), 'Existe algún pago fijo por el lugar')]/parent::li/div/text())").get(),
        FIELD_NAMES[14]: respObj.xpath("normalize-space(//span[contains(text(), 'Cómo se efectúa el pago')]/parent::li/div/text())").get(),
        FIELD_NAMES[15]: driver.current_url
    }
    print(data)
    writeCSV(data, FIELD_NAMES, f'''/content/drive/MyDrive/spain/{baseUrl.split("/")[-1].strip()}.csv''')
    data.clear()

driver.quit()


# Script for Italy Region

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from scrapy import Selector
from selenium_stealth import stealth
import os
import csv
import time


FIELD_NAMES = ['Name', 'Type', 'Address', 'Phone', 'Menù a partire da', 'Nº di invitati', 'Ambienti', 'Servizi', 'Alloggio', 'Cosa comprende per sommi capi il menù', 'È possibile affittare solamente la struttura senza il servizio catering', 'Ci sono delle restrizioni orarie per il ricevimento', 'Il catering è tassato', 'È previsto un costo fisso per laffitto della struttura', 'Come si effettua il pagamento', 'Source Url']
BASE_URLS = [
             'https://www.matrimonio.com/ville-matrimoni',
             'https://www.matrimonio.com/agriturismo-ricevimenti',
             'https://www.matrimonio.com/hotel-ricevimenti',
             'https://www.matrimonio.com/ristoranti-ricevimenti',
             'https://www.matrimonio.com/castelli-matrimoni',
             'https://www.matrimonio.com/matrimoni-in-spiaggia',
             'https://www.matrimonio.com/location-matrimoni',
]

def writeCSV(data, fieldName, FILE_NAME):
    fileExists = os.path.isfile(FILE_NAME)
    with open(FILE_NAME, 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldName, lineterminator='\n')
        if not fileExists:
            writer.writeheader()
        writer.writerow(data)

chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory" : "/some/path"}
# chromeOptions.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

for baseUrl in BASE_URLS:
  urlList = []
  pageCntr = 1
  driver.get(baseUrl)
  time.sleep(2)

  try:
    cookiesBtnElem = driver.find_element_by_xpath("//button[text()='Accetta']")
    driver.execute_script("arguments[0].click()", cookiesBtnElem)
    time.sleep(1)
  except:
    pass

  while True:
    pageCntr += 1
    html = driver.page_source
    respObj = Selector(text=html)

    #if pageCntr > 27:
    cards = respObj.xpath("//div[@data-list-type='Catalog']/div[@id]")
    for card in cards:
      urlList.append(card.xpath(".//a[contains(@id, 'app_lnk')]/@href").get())

    nextPageType1 = respObj.xpath(f"//a[@data-page and text()='{pageCntr}']")
    nextPageType2 = respObj.xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
    
    if nextPageType1:
      nextBtnElem = driver.find_element_by_xpath(f"//a[@data-page and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    elif nextPageType2:
      nextBtnElem = driver.find_element_by_xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    else:
      break

  for url in urlList:
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//h1")))

    try:
      phoneBtnElem = driver.find_element_by_xpath("//span[@class='app-emp-phone-txt']")
      driver.execute_script("arguments[0].click()", phoneBtnElem)
      WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//i[contains(@class, 'phone')]/parent::p")))
    except:
      pass

    html = driver.page_source
    respObj = Selector(text=html)

    quelType = respObj.xpath("//span[contains(text(), 'de cocina sirves')]/parent::li/div/div/div/text()").getall()
    prem = respObj.xpath("//h1/following-sibling::i/@class").get()
    try:
      if "premium" in prem:
        prem = "Premium"
    except:
      pass
    
    data = {
        FIELD_NAMES[0]: respObj.xpath("normalize-space(//h1/text())").get(),
        FIELD_NAMES[1]: prem,
        FIELD_NAMES[2]: respObj.xpath("normalize-space((//div[contains(@class, 'address')]/text())[2])").get(),
        FIELD_NAMES[3]: f'''{respObj.xpath("normalize-space(//i[contains(@class, 'phone')]/parent::p/text()[last()])").get()}''',
        FIELD_NAMES[4]: respObj.xpath("normalize-space(//span[text()='Menù a partire da' or text()='Affitto struttura']/following-sibling::span/text())").get(),
        FIELD_NAMES[5]: respObj.xpath('''normalize-space(//span[text()="Nº di invitati"]/following-sibling::span/text())''').get(),
        FIELD_NAMES[6]: f'''{respObj.xpath("normalize-space(//span[text()='Ambienti']/parent::div/text()[2])").get()}{respObj.xpath("normalize-space(//span[text()='Ambienti']/parent::div/span[@style]/text())").get()}''',
        FIELD_NAMES[7]: respObj.xpath("normalize-space(//span[text()='Servizi']/parent::div/text()[last()])").get(),        
        FIELD_NAMES[8]: respObj.xpath("normalize-space(//span[text()='Alloggio']/parent::div/text()[last()])").get(),
        FIELD_NAMES[9]: respObj.xpath("normalize-space(//span[contains(text(), 'Cosa comprende per sommi capi il menù')]/parent::li/div/text())").get(),
        FIELD_NAMES[10]: respObj.xpath("normalize-space(//span[contains(text(), 'È possibile affittare solamente la struttura')]/parent::li/div/text())").get(),        
        FIELD_NAMES[11]: respObj.xpath("normalize-space(//span[contains(text(), 'Ci sono delle restrizioni orarie per')]/parent::li/div/text())").get(),
        FIELD_NAMES[12]: None,
        FIELD_NAMES[13]: respObj.xpath("normalize-space(//span[contains(text(), 'previsto un costo fisso per')]/parent::li/div/text())").get(),
        FIELD_NAMES[14]: respObj.xpath("normalize-space(//span[contains(text(), 'Come si effettua il pagamento')]/parent::li/div/text())").get(),        
        FIELD_NAMES[15]: driver.current_url
    }
    print(data)
    writeCSV(data, FIELD_NAMES, f'''/content/drive/MyDrive/italy/{baseUrl.split("/")[-1].strip()}.csv''')
    data.clear()
print("Scrapping Finished Successfully!\nShutting down the chromedriver...")
driver.quit()


# Script for UK Region



In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from scrapy import Selector
from selenium_stealth import stealth
import os
import csv
import time


FIELD_NAMES = ['Name', 'Type', 'Address', 'Phone', 'Website Url', 'Capacity', 'Menus From', 'Venue Type', 'Dining Options', 'Evening Entertainment', 'Overnight Accommodation', 'Venue Staff Assistance', 'Venue Leisure Facilities', 'Other Features', 'Source Url']
BASE_URLS = ['https://www.hitched.co.uk/wedding-venues']

def writeCSV(data, fieldName, FILE_NAME):
    fileExists = os.path.isfile(FILE_NAME)
    with open(FILE_NAME, 'a', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldName, lineterminator='\n')
        if not fileExists:
            writer.writeheader()
        writer.writerow(data)

chrome_options = webdriver.ChromeOptions()
# prefs = {"download.default_directory" : "/some/path"}
# chromeOptions.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

for baseUrl in BASE_URLS:
  urlList = []
  pageCntr = 1
  driver.get(baseUrl)
  time.sleep(2)

  try:
    cookiesBtnElem = driver.find_element_by_xpath("//button[text()='Accept']")
    driver.execute_script("arguments[0].click()", cookiesBtnElem)
    time.sleep(1)
  except:
    pass

  while True:
    pageCntr += 1
    html = driver.page_source
    respObj = Selector(text=html)

    #if pageCntr > 27:
    cards = respObj.xpath("//div[@data-list-type='Catalog']/div[@id]")
    for card in cards:
      urlList.append(card.xpath(".//a[contains(@id, 'app_lnk')]/@href").get())

    nextPageType1 = respObj.xpath(f"//a[@data-page and text()='{pageCntr}']")
    nextPageType2 = respObj.xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
    
    if nextPageType1:
      nextBtnElem = driver.find_element_by_xpath(f"//a[@data-page and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    elif nextPageType2:
      nextBtnElem = driver.find_element_by_xpath(f"//span[contains(@class, 'pagination') and text()='{pageCntr}']")
      driver.execute_script("arguments[0].click()", nextBtnElem)
      time.sleep(2)
      print(f"\n\n PAGE-{pageCntr}")
    else:
      break

  for url in urlList:
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//h1")))

    try:
      phoneBtnElem = driver.find_element_by_xpath("//span[@class='app-emp-phone-txt']")
      driver.execute_script("arguments[0].click()", phoneBtnElem)
      WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//i[contains(@class, 'phone')]/parent::p")))
    except:
      pass

    html = driver.page_source
    respObj = Selector(text=html)

    venueType = respObj.xpath("//span[contains(text(), 'Venue Type')]/parent::li/div/div/div/text()").getall()
    diningOptions = respObj.xpath("//span[contains(text(), 'Dining Options')]/parent::li/div/div/div/text()").getall()
    eveningEntertainment = respObj.xpath("//span[contains(text(), 'Evening Entertainment')]/parent::li/div/div/div/text()").getall()
    overnightAccommodation = respObj.xpath("//span[contains(text(), 'Overnight Accommodation')]/parent::li/div/div/div/text()").getall()
    venueStaffAssistance = respObj.xpath("//span[contains(text(), 'Venue Staff Assistance')]/parent::li/div/div/div/text()").getall()
    venueLeisureFacilities = respObj.xpath("//span[contains(text(), 'Venue Leisure Facilities')]/parent::li/div/div/div/text()").getall()
    otherFeatures = respObj.xpath("//span[contains(text(), 'Other Features')]/parent::li/div/div/div/text()").getall()
    
    prem = respObj.xpath("//h1/following-sibling::i/@class").get()
    try:
      if "premium" in prem:
        prem = "Premium"
    except:
      pass
    
    data = {
        FIELD_NAMES[0]: respObj.xpath("normalize-space(//h1/text())").get(),
        FIELD_NAMES[1]: prem,
        FIELD_NAMES[2]: respObj.xpath("normalize-space((//div[contains(@class, 'address')]/text())[2])").get(),
        FIELD_NAMES[3]: f'''{respObj.xpath("normalize-space(//i[contains(@class, 'phone')]/parent::p/text()[last()])").get()}''',      
        FIELD_NAMES[4]: respObj.xpath("//span[contains(text(), 'Website')]/@data-url").get(),
        FIELD_NAMES[5]: respObj.xpath('''normalize-space(//span[contains(text(), 'Capacity') or contains(text(), 'guests')]/parent::div/text()[last()])''').get(),
        FIELD_NAMES[6]: respObj.xpath("normalize-space(//span[contains(text(), 'Menus') or contains(text(), 'Starting')]/parent::div/text()[last()])").get(),
        FIELD_NAMES[7]: ", ".join(i.strip() for i in venueType),
        FIELD_NAMES[8]: ", ".join(i.strip() for i in diningOptions),
        FIELD_NAMES[9]: ", ".join(i.strip() for i in eveningEntertainment),        
        FIELD_NAMES[10]: ", ".join(i.strip() for i in overnightAccommodation),
        FIELD_NAMES[11]: ", ".join(i.strip() for i in venueStaffAssistance),
        FIELD_NAMES[12]: ", ".join(i.strip() for i in venueLeisureFacilities),
        FIELD_NAMES[13]: ", ".join(i.strip() for i in otherFeatures),
        FIELD_NAMES[14]: driver.current_url
    }
    print(data)
    writeCSV(data, FIELD_NAMES, f'''/content/drive/MyDrive/uk/{baseUrl.split("/")[-1].strip()}.csv''')
    data.clear()

driver.quit()


{'Name': 'Rustic & Rural Weddings', 'Type': 'Premium', 'Address': 'Goose Lane, Hawksworth, Bradford\xa0(West Yorkshire) LS20 8PE', 'Phone': '+4407946851538', 'Website Url': 'http://www.rusticandruralweddings.com', 'Capacity': '30 to 200', 'Menus From': '£40', 'Venue Type': 'Exclusive Use, In the Country, Sporting, Coastal Weddings, Unusual Weddings, Landmark, Marquees and Tents, Woodland and Festival, Tipis and Yurts, Unique Weddings', 'Dining Options': 'Seated Meal Facilities, Alcohol Licence, Buffet Meal Facilities, In House Catering', 'Evening Entertainment': 'Marquee Available, Dance Floor, Entertainment Licence, Live Band Facilities, Evening Reception Facilities, DJ Facilities, Outdoor Fireworks Permitted', 'Overnight Accommodation': 'Overnight Accommodation for Wedding Couple, Overnight Accommodation for Guests', 'Venue Staff Assistance': 'Master of Ceremonies/ Toastmaster, Wedding Coordinator', 'Venue Leisure Facilities': 'Hairdresser, Beautician', 'Other Features': 'Allows fire

# Command to run scrapy spider through GOOGLE COLAB

In [ ]:
!pip install scrapy
!cd /content/drive/MyDrive/finnhubAPI ; scrapy crawl finnhub